In [ ]:
import pandas as pd
from time import sleep
import re
import threading
import random
from selenium import webdriver # 从selenium导入webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import os
from itertools import product

options = webdriver.ChromeOptions()
with open('万方-统计学/succeed.txt','r',encoding='utf-8') as f:
    data = f.readlines()
succeed = [x.strip() for x in data] #记录完成的名单
mylock = threading.Lock() #给下载成功的名单加锁
def get_info(colleges,magazines):
    global options
    global succeed
    #用笛卡尔积替代双重for循环，稳
    random.shuffle(colleges) #随机打乱
    random.shuffle(magazines)
    user_agent_list = [ \
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1" \
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36 OPR/26.0.1656.60" \
        "Opera/8.0 (Windows NT 5.1; U; en)"\
        "Mozilla/5.0 (Windows NT 5.1; U; en; rv:1.8.1) Gecko/20061208 Firefox/2.0.0 Opera 9.50" \
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; en) Opera 9.50" \
        "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:34.0) Gecko/20100101 Firefox/34.0" \
        "Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10" \
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER" \
        "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; LBBROWSER)" \
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E; LBBROWSER)" \
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 SE 2.X MetaSr 1.0" \
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; SE 2.X MetaSr 1.0)" \
    ]
    url = 'http://www.wanfangdata.com.cn/searchResult/getAdvancedSearch.do?searchType=all'
    for college,magazine in product(colleges,magazines):
        failed_count = 0 #用于记录失败次数，暂时休息应对反爬
        the_item = college+'-'+magazine
        if the_item in succeed:
            #print(the_item+' 已完成.')
            continue
        isExists = os.path.exists('万方-统计学' + '/' + college) #创建学者文件夹
        if not isExists:
            os.makedirs('万方-统计学' + '/' + college)
        UserAgent = 'user-agent=' + random.choice(user_agent_list)
        options.add_argument(UserAgent)
        index_path = 'D:\\codes\\GitProject\\工作区\\首经贸数据采集项目\\万方-统计学'
        college_path = index_path + '\\' + college
        prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': college_path}  #下载路径
        options.add_experimental_option('prefs', prefs)
        driver = webdriver.Chrome(chrome_options=options)
        try:  #访问
            driver.get(url)
        except:
            try:
                driver.refresh()
            except:
                print('超时错误')
                driver.quit()
                continue
        sleep(2)
        try:
            driver.find_element_by_id('cancel_all').click() #清除所有
            driver.find_elements_by_name('thesis_type')[0].click() #期刊论文

            Select(driver.find_elements_by_name('search_field')[1]).select_by_value('期刊—ISSN/CN')
            mag = driver.find_elements_by_id('ddd')[1]   
            mag.send_keys(magazine)
            Select(driver.find_elements_by_name('vague_accurate')[1]).select_by_value('accurate')#精确查找
            col = driver.find_elements_by_id('ddd')[2]
            col.send_keys(college)
            search = driver.find_element_by_id('set_advanced_search_btn')
            search.click() #点击查找
            sleep(8)
            page = driver.page_source
            if page.find('没有命中的记录，请重试或修改检索表达式') > -1:
                print(the_item + '  没有检索结果')
                succeed.append(the_item)
                driver.quit()
                continue
            
            while True:
                driver.find_element_by_name('checkAll').click() #全选
                sleep(5)
                try:
                    driver.find_element_by_link_text(u'下一页').click()
                    sleep(8)
                except:
                    #print('没有下一页')
                    sleep(1)
                    break
            #导出前进行判断，是否把所有的检索结果都选中了
            result = driver.find_element_by_class_name('result_stati').text
            m1 = re.search('\d+',result)
            result_counts = m1.group(0)
            my_result = driver.find_element_by_name('checkAll').text
            m2 = re.search('\d+',my_result)
            my_counts = m2.group(0)
            if result_counts != my_counts:
                print(scholar +' 检索结果:' + str(result_counts) + ' 选中结果:' + str(my_counts))
                print('检索未全部选中，本次作废')
                driver.quit()
                sleep(4)
                continue
            #批量导出
            driver.find_element_by_link_text(u'批量导出').click()
            driver.switch_to_window(driver.window_handles[1]) #可能要重新定位windows
            loc2 = (By.ID,'export')
            ele2 = WebDriverWait(driver,10).until(EC.presence_of_element_located(loc2))
            driver.find_element_by_link_text(u'自定义格式').click()
            sleep(5)
            driver.find_element_by_class_name('export_excel').click()
            print( the_item + ' 抓取成功')
            sleep(15)
            succeed.append(the_item)
            succeed = list(set(succeed))
            with open('经济学/succeed.txt','w',encoding='utf-8') as f:
                for item in succeed:
                    f.write(item)
                    f.write('\n')
            if len(set(succeed)) == 64:
                print('统计学（统计学）-南京大学C刊全部下载完毕。')
                break
            driver.quit()
        except:
            print(the_item+' 抓取失败')
            failed_count += 1
            if failed_count == 3: #如果连续3次失败，则跳出循环
                driver.quit()
                sleep(150)
            else:
                sleep(5)
                driver.quit()
def to_download(counts):
    data = pd.read_excel('高校名单.xlsx',sheet_name='统计学')
    colleges = data['高校名称']
    data = pd.read_excel('南京大学c刊.xlsx',sheet_name='统计学')
    magazines = data['期刊名称']
    for i in range(counts):
        print('第'+str(i+1)+'次循环')
        get_info(colleges,magazines)

if __name__ == '__main__':
    counts = 10
    threads = []
    for i in range(4):
        t = threading.Thread(target=to_download,args=(counts,))
        threads.append(t)
    for t in threads:
        t.start()
    for thread in threads:
        thread.join()

第1次循环
第1次循环
第1次循环
第1次循环
